In [11]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
from urllib.parse import urljoin
import re
import json
from datetime import datetime

In [12]:
class MarmitonScraper:
    def __init__(self):
        self.base_url = "https://www.marmiton.org"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }
        self.session = requests.Session()
        self.session.headers.update(self.headers)

    def get_recipe_urls(self, category="plat-principal", max_pages=3):
        urls = []
        for page in range(1, max_pages + 1):
            try:
                search_url = f"{self.base_url}/recettes/recherche.aspx?aqt={category}&page={page}"
                response = self.session.get(search_url, timeout=10)
                soup = BeautifulSoup(response.content, 'html.parser')
                links = soup.find_all('a', href=re.compile(r'/recettes/recette_'))
                
                page_urls = [urljoin(self.base_url, link.get('href')) for link in links if link.get('href')]
                urls.extend(page_urls)
                print(f"✓ Page {page} - {len(page_urls)} URLs trouvées")
                time.sleep(random.uniform(1, 2))
            except Exception as e:
                print(f"❌ Erreur page {page}: {e}")
        return list(set(urls))

    def extract_recipe_details(self, url):
        try:
            response = self.session.get(url, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extraction des badges (Temps, Difficulté, Budget)
            badges = [b.text.strip().upper() for b in soup.select('.recipe-primary__item')]
            
            # Initialisation des variables
            diff = "MOYEN"
            budget = "NON SPÉCIFIÉ"
            t_prep = 0
            t_cuis = 0

            # Attribution intelligente (au lieu des index fixes)
            for b in badges:
                if "MIN" in b or "H" in b:
                    if "PRÉP" in b.lower(): t_prep = self._parse_minutes(b)
                    elif "CUIS" in b.lower(): t_cuis = self._parse_minutes(b)
                    elif t_prep == 0: t_prep = self._parse_minutes(b) # Fallback
                if any(x in b for x in ["FACILE", "MOYEN", "DIFFICILE"]):
                    diff = b.replace("TRÈS ", "")
                if any(x in b for x in ["BON MARCHÉ", "COÛTEUX"]):
                    budget = b

            title = self._extract_title(soup)
            
            return {
                'url': url,
                'titre': title,
                'description': self._extract_description(soup),
                'difficulte': diff,
                'budget': budget,
                'temps_preparation': t_prep,
                'temps_cuisson': t_cuis,
                'type_recette': self._classify_type(title),
                'cuisine': self._classify_cuisine(title),
                'ingredients': self._extract_ingredients(soup), # Garder en liste
                'etapes': self._extract_steps(soup),           # Garder en liste
                'date_scraping': datetime.now().isoformat(),
                'imageUrl': self._extract_image(soup)
            }
        except Exception as e:
            print(f"❌ Erreur sur {url}: {e}")
            return None

    def _parse_minutes(self, text):
        """Convertit '1h20' ou '20 min' en minutes (int)"""
        text = text.upper()
        digits = re.findall(r'\d+', text)
        if not digits: return 0
        if 'H' in text:
            hours = int(digits[0])
            mins = int(digits[1]) if len(digits) > 1 else 0
            return (hours * 60) + mins
        return int(digits[0])

    # --- NOUVELLES MÉTHODES D'EXTRACTION ---

    def _extract_description(self, soup):
        # Cherche la meta description ou le texte d'introduction
        desc = soup.find('meta', property='og:description')
        if desc: return desc['content'].strip()
        # Fallback sur le texte de présentation si dispo
        summary = soup.select_one('.recipe-content__description')
        return summary.text.strip() if summary else ""

    def _extract_badge_info(self, soup, index):
        # Marmiton utilise des classes type "recipe-primary__item" pour difficulté/budget
        badges = soup.select('.recipe-primary__item')
        try:
            if badges and len(badges) > index:
                return badges[index].text.strip().upper()
        except:
            pass
        return "NON SPÉCIFIÉ"

    def _extract_time(self, soup, type_time):
        # Cherche dans les éléments de temps (ex: "15 min")
        time_elements = soup.select('.recipe-primary__item')
        for el in time_elements:
            text = el.text.lower()
            if type_time in text:
                # Extraction du nombre via regex
                numbers = re.findall(r'\d+', text)
                return int(numbers[0]) if numbers else 0
        return 0

    def _extract_image(self, soup):
        img = soup.find('meta', property='og:image')
        return img['content'] if img else ""

    def _extract_title(self, soup):
        title = soup.find('h1')
        return title.text.strip() if title else "Sans titre"

    def _classify_type(self, title):
        title = title.lower()
        if any(x in title for x in ['tarte', 'gâteau', 'mousse', 'dessert']): return 'dessert'
        if any(x in title for x in ['entrée', 'salade', 'soupe']): return 'entree'
        return 'plat'

    def _classify_cuisine(self, title):
        title = title.lower()
        if 'pizza' in title or 'pasta' in title: return 'italienne'
        return 'française'

    def _extract_ingredients(self, soup):
        ingredients = []
        # Sélecteur spécifique à la nouvelle version de Marmiton
        items = soup.select('.card-ingredient')
        for item in items:
            name = item.select_one('.ingredient-name')
            if name: ingredients.append({"nom": name.text.strip()})
        return ingredients

    def _extract_steps(self, soup):
        items = soup.select('div[class*="RecipeStep"]')
        return [item.get_text().strip() for item in items]

    def scrape_all(self, categories=['plat-principal'], max_pages=1):
        all_results = []
        for cat in categories:
            print(f"\n📂 Catégorie : {cat}")
            urls = self.get_recipe_urls(cat, max_pages)
            for i, url in enumerate(urls, 1):
                print(f"  [{i}/{len(urls)}] Scraping...")
                data = self.extract_recipe_details(url)
                if data: all_results.append(data)
                time.sleep(random.uniform(0.5, 1.5))
        return all_results

    def save_to_csv(self, recipes, filename='marmiton_v2.csv'):
        if not recipes:
            print("⚠ Aucune donnée collectée. Le fichier n'a pas été créé.")
            return pd.DataFrame() # On renvoie un DataFrame vide plutôt que None
        
        df = pd.DataFrame(recipes)
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"\n✅ {len(df)} recettes sauvegardées dans {filename}")
        return df

if __name__ == "__main__":
    scraper = MarmitonScraper()
    print("🚀 Démarrage du scraping intensif Marmiton...")
    
    # MODIFICATION ICI :
    # 3 catégories x 4 pages (~15 recettes/page) = ~180 URLs potentielles
    # On vise large car certaines recettes peuvent échouer ou être des doublons
    recipes = scraper.scrape_all(
        categories=['plat-principal', 'entree', 'dessert'],
        max_pages=6  # Augmentez à 5 ou 6 si vous voulez être sûr d'avoir 140+
    )
    
    # Sauvegarde
    df = scraper.save_to_csv(recipes, filename='marmiton_v2.csv')
    
    # Vérification et statistiques
    if df is not None and not df.empty:
        print(f"\n📊 Statistiques finales :")
        print(f"  - Total recettes collectées : {len(df)}")
        
        if len(df) < 140:
            print(f"  ⚠️ Attention : Vous n'avez que {len(df)} recettes. Relancez avec plus de max_pages.")
        else:
            print(f"  ✅ Objectif atteint : {len(df)} recettes prêtes pour l'importation.")
            
        if 'type_recette' in df.columns:
            print(f"  - Répartition : {df['type_recette'].value_counts().to_dict()}")
    else:
        print("\n❌ Échec du scraping : aucune donnée récupérée.")

🚀 Démarrage du scraping intensif Marmiton...

📂 Catégorie : plat-principal
✓ Page 1 - 21 URLs trouvées
✓ Page 2 - 21 URLs trouvées
✓ Page 3 - 21 URLs trouvées
✓ Page 4 - 21 URLs trouvées
✓ Page 5 - 21 URLs trouvées
✓ Page 6 - 21 URLs trouvées
  [1/96] Scraping...
  [2/96] Scraping...
  [3/96] Scraping...
  [4/96] Scraping...
  [5/96] Scraping...
  [6/96] Scraping...
  [7/96] Scraping...
  [8/96] Scraping...
  [9/96] Scraping...
  [10/96] Scraping...
  [11/96] Scraping...
  [12/96] Scraping...
  [13/96] Scraping...
  [14/96] Scraping...
  [15/96] Scraping...
  [16/96] Scraping...
  [17/96] Scraping...
  [18/96] Scraping...
  [19/96] Scraping...
  [20/96] Scraping...
  [21/96] Scraping...
  [22/96] Scraping...
  [23/96] Scraping...
  [24/96] Scraping...
  [25/96] Scraping...
  [26/96] Scraping...
  [27/96] Scraping...
  [28/96] Scraping...
  [29/96] Scraping...
  [30/96] Scraping...
  [31/96] Scraping...
  [32/96] Scraping...
  [33/96] Scraping...
  [34/96] Scraping...
  [35/96] Scraping